In [1]:
import torch
import pandas as pd

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [12]:
phrase = "My friend is right about the class , it was so fun"

In [13]:
encoded_phrase = tokenizer(phrase, return_tensors = 'pt')

In [14]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  318,  826,  546,  262, 1398,  837,  340,  373,  523, 1257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
response = model(**encoded_phrase, output_attentions = True , output_hidden_states = True)

In [16]:
print(len(response.attentions))

12


In [17]:
response.attentions[1].shape  #from final decoder
# batch size - 1
# multi heads - 12 -in final enc
# 12 tokens in input / output -Mt friend is right about the class , it was so fun

torch.Size([1, 12, 12, 12])

In [18]:
#convert if into tokens 
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
tokens #G in front corresponds to space 

['My',
 'Ġfriend',
 'Ġis',
 'Ġright',
 'Ġabout',
 'Ġthe',
 'Ġclass',
 'Ġ,',
 'Ġit',
 'Ġwas',
 'Ġso',
 'Ġfun']

In [19]:
arr = response.attentions[9][0][0] #grab attention from 10 th econder which is 9th index . first attention [0] . batch size 1 [0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10 **n_digits) / (10 **n_digits)).detach()).applymap(float)

In [20]:
attention_df.columns = tokens
attention_df.index = tokens

In [21]:
attention_df

,My,Ġfriend,Ġis,Ġright,Ġabout,Ġthe,Ġclass,"Ġ,",Ġit,Ġwas,Ġso,Ġfun
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġis,0.876,0.087,0.037,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.980,0.007,0.006,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.006,0.003,0.005,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthe,0.934,0.014,0.008,0.013,0.015,0.016,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.928,0.003,0.002,0.001,0.001,0.002,0.063,0.000,0.000,0.000,0.000,0.000
"Ġ,",0.820,0.009,0.006,0.005,0.006,0.007,0.141,0.007,0.000,0.000,0.000,0.000
Ġit,0.513,0.005,0.004,0.010,0.010,0.011,0.387,0.029,0.033,0.000,0.000,0.000
Ġwas,0.259,0.012,0.003,0.010,0.005,0.007,0.582,0.014,0.046,0.061,0.000,0.000


In [22]:
from bertviz import model_view

In [23]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [24]:
tokens

['My',
 'Ġfriend',
 'Ġis',
 'Ġright',
 'Ġabout',
 'Ġthe',
 'Ġclass',
 'Ġ,',
 'Ġit',
 'Ġwas',
 'Ġso',
 'Ġfun']

In [29]:
tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])

['Ċ',
 ',',
 'Ġa',
 '.',
 'Ġthat',
 'Ġfact',
 '.',
 'Ġbut',
 "'s",
 'Ġa',
 'Ġhard',
 'Ġto']

In [30]:
df = pd.DataFrame(zip(tokens,tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
                 columns = ["current" , "next"])

In [31]:
df

,current,next
0,My,Ċ
1,Ġfriend,","
2,Ġis,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthe,Ġfact
6,Ġclass,.
7,"Ġ,",Ġbut
8,Ġit,'s
9,Ġwas,Ġa


In [ ]:
gene